# ✅ TOTP 구조 (Time-based One-Time Password)

> TOTP는 시간(time)을 기반으로 OTP를 생성
>
>
> **30초마다 바뀌는 OTP**, 실무에 가장 많이 쓰임 (구글 OTP 앱 등)
>

---

### 🔹 핵심 공식

```
TOTP = Truncate(HMAC(K, T))
       → K: 공유된 시크릿 키 (Base32)
       → T: 시간값 (timestamp / 30)

```

- `timestamp // 30` → **30초 단위 블록**
- `HMAC-SHA1` 또는 SHA256 사용
- 최종 OTP는 Truncate(마지막 몇 자 잘라냄)

---

### 🔹 Python 코드 예시

In [1]:
import pyotp

secret = pyotp.random_base32()
totp = pyotp.TOTP(secret)

print("Secret Key:", secret)
print("현재 OTP:", totp.now())



Secret Key: BOKME35CELAOPVD7EK4KZPNIXLXSQ5BY
현재 OTP: 814325


---

### 🔹 서버 검증 흐름

1. 서버와 클라이언트가 **동일한 secret key** 공유
2. 클라이언트는 현재 시간 기준으로 OTP 생성
3. 서버는 **자기 시간 기준 OTP와 비교**

    → 보통 ±1 블록(30초) 오차 허용


---

## ✅ 동기화 방식 vs 비동기화 방식 비교

| 구분 | 🔵 동기화 방식 | 🔴 비동기화 방식 |
| --- | --- | --- |
| 개념 | 서버-클라이언트 간 **시간 또는 카운터 일치 필요** | 미리 생성된 해시값을 **역순으로** 사용 |
| 대표 방식 | TOTP (시간 동기화)HOTP (이벤트 동기화) | S/KEY |
| 사용 예 | 구글 OTP, MS Authenticator | 구형 시스템, 연구용 |
| 서버 저장 | 시크릿 키 + 현재 시간 or 카운터 | 마지막 사용한 해시값 |
| 공격 방지 | OTP 재사용 불가시간 지남에 따라 무효화 | 해시체인 역순 → 재사용 차단 |
| 실무 사용성 | ✅ 일반적, 많이 쓰임 | ❌ 드묾, 단순한 구조 |
| 시험 출제 포인트 | “시간 동기화 방식은?”“비동기화 방식은?” | “해시 체인 기반은?” |

---

### ✅ 이건 꼭 외워야 돼요

| 방식 | 분류 | 핵심 |
| --- | --- | --- |
| 🔸 S/KEY | 비동기 | 해시 체인 사용, 동기화 불필요 |
| 🔸 HOTP | 동기화 | 이벤트 기반 카운터 동기화 |
| 🔸 TOTP | 동기화 | 시간 기반 30초 단위 동기화 |

---

## ✅ 실무자용 한줄 요약

> 🔹 TOTP는 시간 동기화 방식이며, HMAC(secret, timestamp // 30) 구조로 OTP 생성
>
>
> 🔹 **S/KEY는 해시체인 기반 비동기 방식**, 매번 역순 해시로 인증
>

## 예시 문제

- 동기화 방식 OTP에서는 시간과 인증 횟수를 기반으로 비밀번호를 동기화한다.
- 비동기화 방식 OTP에서는 인증서버에서 전송된 난수를 기반으로 비밀번호를 생성한다.
- 시간 동기화 방식 OTP는 인증서버와 OTP생성기의 시간오차범위를 허용하지 않는다. (X)

---

## ✅ 보기 해석과 실무 감각 대응

---

### 🔸 보기 1

**"동기화 방식 OTP에서는 시간과 인증 횟수를 기반으로 비밀번호를 동기화한다."**

🔍 **직관 해석**:

- "시간" → TOTP
- "인증 횟수" → HOTP

    → **두 개를 묶어 말했지만**, 사실은 **각각 따로 쓰이는 기준**임


✅ 실무적 정리:

- HOTP는 **이벤트(횟수)** 기반
- TOTP는 **시간(30초 단위)** 기반

    → 둘 다 동기화 방식이긴 하지만, **기준은 다름**

    → 말장난 주의!


> ✅ 시험에서는 "시간 또는 인증 횟수를 기준으로 동기화함"이면 맞는 보기로 처리돼요.
>

---

### 🔸 보기 2

**"비동기화 방식 OTP에서는 인증서버에서 전송된 난수를 기반으로 비밀번호를 생성한다."**

🔍 **실무 감각**:

- 난수 전송해서 만드는 건 **OTP 토큰이 아니라** **Challenge-Response 방식**에 가까움
- S/KEY는 서버가 **난수 안줌**, 클라이언트가 해시 체인을 미리 계산해서 갖고 있음

❌ 이 보기 → 틀린 이유:

- **비동기화 방식은 미리 만들어진 해시 체인**을 쓰는 방식 (ex. S/KEY)
- 서버에서 "난수 전송"은 없어요

> ❗️ "난수 기반 OTP"는 다른 인증 방식(Challenge-Response 방식)과 헷갈리게 만든 보기예요.
>

---

### 🔸 보기 3

**"시간 동기화 방식 OTP는 인증서버와 OTP생성기의 시간오차범위를 허용하지 않는다."** ← ❌

🔍 **실무 감각**:

- 서버와 클라이언트가 완벽히 시간 일치할 수 없음

    → 대부분의 구현에서 **±30초 또는 ±1 블록 정도 허용함**


✅ TOTP에서는 보통:

```python
totp.verify(otp, valid_window=1)
```

→ ±1 window(30초 오차) 허용

❌ 이 보기의 문제:

- "**오차범위를 허용하지 않는다**" → 현실적이지 않음
- 실제 서버는 항상 오차를 허용함 (시계 drift 대비)

---

## ✅ 실무자용 암기법 (시험 대비 압축 요약)

| 보기 요소 | 핵심 정리 |
| --- | --- |
| ✅ 동기화 | HOTP(횟수), TOTP(시간) 기준으로 OTP 동기화 |
| ❌ 비동기에서 난수 전송? | X. S/KEY는 해시체인. 서버에서 난수 안줌 |
| ❌ TOTP는 시간 오차 허용 X? | X. 현실에선 오차 30초~1분 허용함 |

---

## ✅ 현실적인 한줄 정리

> ✅ TOTP는 ±30초 정도의 시간 오차를 허용해야 정상 작동하고,
>
>
> ✅ **S/KEY는 해시체인을 미리 만들어 쓰는 비동기 구조**이며,
>
> ❌ **"서버가 난수 보내는 비동기"는 말장난이다.**
>

## OTP 와 대칭키 공유

## ✅ 보기:

**“OTP는 두 사용자가 사전에 대칭키를 공유해야 한다.”** → **⭕ 맞는 말**

---

## ✅ 실무자 감각 해석

- 이게 **말은 어렵게 썼지만**, 사실 그냥

    👉 **“OTP 생성기와 서버가 같은 시크릿 키를 갖고 있어야 한다”**는 얘기예요.


---

## ✅ 구체적으로:

| 방식 | 공유 키 존재 여부 | 공유 방식 |
| --- | --- | --- |
| 🔸 TOTP | ✅ 있음 | Base32 secret key를 앱과 서버가 공유 |
| 🔸 HOTP | ✅ 있음 | 마찬가지로 키 공유 필요 |
| 🔸 S/KEY | ❌ 없음 | 해시체인 사용. 키 공유 아님 |

즉,

> 🔐 TOTP, HOTP는 HMAC 기반이므로 반드시 공유된 “대칭 키”가 필요
>

---

## ✅ 실무 감각 요약

- “OTP = 공유된 시크릿 + 시간(or 카운터) 기반 코드 생성”
- **그 시크릿이 바로 대칭키**
- 실무에선 QR코드 스캔할 때 이미 앱에 **secret key가 embed**됨

---

## ✅ 시험 보기 패턴 조심

| 보기 문장 | 실무 감각 | 정답 처리 |
| --- | --- | --- |
| “OTP는 대칭키 기반이다” | → 맞음 (HMAC 기반) | ⭕ |
| “OTP는 키 교환 없이 사용 가능하다” | → ❌ (사전 키 필요) | ❌ |
| “OTP는 공개키 암호 기반이다” | → ❌ | ❌ |

---

## ✅ 정리: 실무자는 이렇게 외우면 끝

> 🔹 OTP는 HMAC 기반 = 대칭키 기반
>
>
> 🔹 서버와 클라이언트가 **시크릿 키를 미리 공유해야 함**
>
> 🔹 **공개키 암호랑은 전혀 관련 없음**
>